In [ ]:
# Save FIPS-enhanced maps and generate summary report
print("Saving FIPS-enhanced maps...")

# Save FIPS-based maps
if fips_data_loaded:
    fips_map_path = os.path.join(output_dir, 'linknyc_fips_choropleth.html')
    m_fips.save(fips_map_path)
    print(f"✓ FIPS county choropleth saved: {fips_map_path}")

comprehensive_map_path = os.path.join(output_dir, 'linknyc_comprehensive_fips_map.html')
m_comprehensive.save(comprehensive_map_path)
print(f"✓ Comprehensive FIPS map saved: {comprehensive_map_path}")

# Create FIPS mapping summary
print("\n" + "="*70)
print("FIPS-ENHANCED CHOROPLETH MAPPING SUMMARY")
print("="*70)

print("\n🗺️  GEOGRAPHIC STANDARDS IMPLEMENTED:")
print("─" * 50)
print("• Official FIPS county codes for NYC boroughs")
print("• Census Bureau 2021 cartographic boundaries")
print("• ZIP Code Tabulation Areas (ZCTA) integration")
print("• Standardized geographic referencing system")

if fips_data_loaded:
    print("\n📍 FIPS COUNTY MAPPING:")
    print("─" * 30)
    for _, county in nyc_counties_with_data.iterrows():
        print(f"• {county['BOROUGH']:<15} → FIPS {county['GEOID']} ({county['NAME']} County)")
        print(f"  {county['total_kiosks']:>3} kiosks, {county['live_percentage']:>5.1f}% operational")

if zcta_data_loaded:
    print(f"\n📮 ZIP CODE ANALYSIS:")
    print("─" * 25)
    print(f"• {len(nyc_zctas)} ZIP codes identified in NYC area")
    if zip_data_available:
        print(f"• {len(zip_stats)} ZIP codes contain LinkNYC kiosks")
        print("• Top ZIP codes by kiosk density integrated")
    else:
        print("• ZIP code kiosk data not available in dataset")

print(f"\n🎯 MAPPING FEATURES:")
print("─" * 20)
print("• Multi-scale geographic analysis (County + ZIP)")
print("• Interactive layer controls and toggles")
print("• Official Census boundary precision")
print("• Status-coded kiosk point markers")
print("• Comprehensive popup information")
print("• Professional cartographic styling")

print(f"\n📂 FILES GENERATED:")
print("─" * 20)
file_list = []
if fips_data_loaded:
    file_list.append("linknyc_fips_choropleth.html - County-level FIPS choropleth")
file_list.append("linknyc_comprehensive_fips_map.html - Multi-layer FIPS dashboard")

for i, file_desc in enumerate(file_list, 1):
    print(f"{i}. {file_desc}")

print(f"\n✅ FIPS integration complete! The choropleth now uses:")
print("   • Standardized FIPS county codes")  
print("   • Official Census Bureau boundaries")
print("   • Multi-level geographic precision")
print("   • Professional cartographic standards")

## Export FIPS-Enhanced Maps

In [ ]:
# Create comprehensive FIPS-based mapping dashboard
print("Creating comprehensive FIPS mapping dashboard...")

# Create a multi-layer map with both county and ZIP code boundaries
m_comprehensive = folium.Map(
    location=nyc_center,
    zoom_start=10,
    tiles='CartoDB positron'
)

# Add multiple base map options
folium.TileLayer('OpenStreetMap').add_to(m_comprehensive)
folium.TileLayer('CartoDB dark_matter', name='Dark Mode').add_to(m_comprehensive)

if fips_data_loaded:
    # County-level choropleth layer
    county_layer = folium.FeatureGroup(name='County Boundaries (FIPS)', show=True)
    
    folium.Choropleth(
        geo_data=nyc_counties_json,
        data=borough_data_fips,
        columns=['GEOID', 'total_kiosks'],
        key_on='feature.properties.GEOID',
        fill_color='Blues',
        fill_opacity=0.4,
        line_opacity=0.8,
        line_color='navy',
        line_weight=3,
        legend_name='Kiosks per County (FIPS)'
    ).add_to(county_layer)
    
    county_layer.add_to(m_comprehensive)

# Add ZIP code boundaries if available
if zcta_data_loaded and zip_data_available:
    zip_layer = folium.FeatureGroup(name='ZIP Code Boundaries', show=False)
    
    # Merge ZIP stats with ZCTA boundaries
    nyc_zctas_with_data = nyc_zctas.merge(
        zip_stats.reset_index().rename(columns={zip_col: 'ZCTA5CE20'}),
        on='ZCTA5CE20',
        how='left'
    )
    
    # Fill NaN values for ZIPs without kiosks
    nyc_zctas_with_data['total_kiosks'] = nyc_zctas_with_data['total_kiosks'].fillna(0)
    nyc_zctas_with_data['live_kiosks'] = nyc_zctas_with_data['live_kiosks'].fillna(0)
    
    # Create ZIP code choropleth
    zip_json = nyc_zctas_with_data.to_crs('EPSG:4326').__geo_interface__
    
    folium.Choropleth(
        geo_data=zip_json,
        data=nyc_zctas_with_data,
        columns=['ZCTA5CE20', 'total_kiosks'],
        key_on='feature.properties.ZCTA5CE20',
        fill_color='Reds',
        fill_opacity=0.6,
        line_opacity=0.5,
        line_color='darkred',
        line_weight=1,
        legend_name='Kiosks per ZIP Code'
    ).add_to(zip_layer)
    
    zip_layer.add_to(m_comprehensive)

# Add kiosk points layer
kiosk_layer = folium.FeatureGroup(name='Individual Kiosks', show=False)
sample_kiosks = mapping_data.sample(n=min(300, len(mapping_data)), random_state=42)

for _, kiosk in sample_kiosks.iterrows():
    color = status_colors.get(kiosk['STATUS'], 'gray')
    folium.CircleMarker(
        [kiosk['LATITUDE'], kiosk['LONGITUDE']],
        radius=2,
        color=color,
        fill=True,
        opacity=0.7,
        popup=f"Status: {kiosk['STATUS']}<br>Borough: {kiosk['BOROUGH']}"
    ).add_to(kiosk_layer)

kiosk_layer.add_to(m_comprehensive)

# Add comprehensive legend
comprehensive_legend = '''
<div style="position: fixed; 
     top: 10px; right: 10px; width: 300px; height: 200px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:12px; padding: 10px">
<h4 style="margin: 0;">NYC LinkNYC FIPS Analysis</h4>
<hr>
<b>Geographic Standards:</b><br>
• County FIPS: Official Census county codes<br>
• ZIP ZCTA: Census ZIP Code Tabulation Areas<br>
• Boundaries: 2021 Census Cartographic Files<br>
<br>
<b>Data Integration:</b><br>
• Kiosk locations mapped to FIPS geographies<br>
• Standardized boundary definitions<br>
• Multi-scale geographic analysis<br>
</div>
'''
m_comprehensive.get_root().html.add_child(folium.Element(comprehensive_legend))

# Add layer control
folium.LayerControl(position='bottomleft').add_to(m_comprehensive)

print("✓ Comprehensive FIPS mapping dashboard created")
print("✓ Includes official Census Bureau boundaries")
print("✓ Multi-scale analysis: County and ZIP code levels")
print("✓ Interactive layer controls for different views")

# Display the comprehensive map
m_comprehensive

In [ ]:
# Load ZIP Code Tabulation Areas (ZCTA) for finer geographic detail
print("Loading ZIP Code boundaries for detailed analysis...")

# Check if we have ZIP code data in the kiosk dataset
zip_columns = [col for col in df.columns if 'ZIP' in col.upper() or 'POSTAL' in col.upper()]
print(f"ZIP code related columns found: {zip_columns}")

# Load ZCTA boundaries from Census
zcta_url = "https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_zcta520_500k.zip"

try:
    print("Downloading ZIP Code Tabulation Area boundaries...")
    
    # Download ZCTA data
    temp_zip_zcta = tempfile.NamedTemporaryFile(delete=False, suffix='.zip')
    urllib.request.urlretrieve(zcta_url, temp_zip_zcta.name)
    
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(temp_zip_zcta.name, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        # Find and read the shapefile
        zcta_shp_files = [f for f in os.listdir(temp_dir) if f.endswith('.shp')]
        if zcta_shp_files:
            zcta_path = os.path.join(temp_dir, zcta_shp_files[0])
            
            print("Reading ZCTA shapefile...")
            zcta_gdf = gpd.read_file(zcta_path)
            
            # Filter for NYC area ZIP codes (approximate bounding box)
            nyc_bbox = {
                'min_lon': -74.5,
                'max_lon': -73.3, 
                'min_lat': 40.4,
                'max_lat': 40.95
            }
            
            # Convert to geographic CRS for bbox filtering
            zcta_gdf = zcta_gdf.to_crs('EPSG:4326')
            
            # Filter ZCTAs within NYC bounding box
            zcta_bounds = zcta_gdf.bounds
            nyc_zctas = zcta_gdf[
                (zcta_bounds['minx'] >= nyc_bbox['min_lon']) &
                (zcta_bounds['maxx'] <= nyc_bbox['max_lon']) &
                (zcta_bounds['miny'] >= nyc_bbox['min_lat']) &
                (zcta_bounds['maxy'] <= nyc_bbox['max_lat'])
            ].copy()
            
            print(f"✓ Found {len(nyc_zctas)} ZIP codes in NYC area")
            
            # If we have ZIP codes in our kiosk data, aggregate by ZIP
            if zip_columns and len(zip_columns) > 0:
                zip_col = zip_columns[0]  # Use first ZIP column found
                print(f"Using ZIP code column: {zip_col}")
                
                # Aggregate kiosks by ZIP code
                zip_data = mapping_data.copy()
                if zip_col in df.columns:
                    zip_data[zip_col] = df.loc[zip_data.index, zip_col]
                    
                    zip_stats = zip_data.groupby(zip_col).agg({
                        'LATITUDE': 'count',
                        'STATUS': lambda x: (x == 'Live').sum(),
                        'BOROUGH': lambda x: x.mode()[0] if len(x) > 0 else 'Unknown'
                    }).rename(columns={
                        'LATITUDE': 'total_kiosks',
                        'STATUS': 'live_kiosks'
                    })
                    
                    zip_stats['live_percentage'] = (zip_stats['live_kiosks'] / zip_stats['total_kiosks'] * 100).round(1)
                    zip_stats = zip_stats[zip_stats['total_kiosks'] > 0]  # Only ZIPs with kiosks
                    
                    print(f"✓ Aggregated data for {len(zip_stats)} ZIP codes with kiosks")
                    print(f"Top 5 ZIP codes by kiosk count:")
                    top_zips = zip_stats.nlargest(5, 'total_kiosks')
                    for zip_code, data in top_zips.iterrows():
                        print(f"• {zip_code}: {data['total_kiosks']} kiosks ({data['live_percentage']:.1f}% live)")
                    
                    zip_data_available = True
                else:
                    print("ZIP code column not found in mapping data")
                    zip_data_available = False
            else:
                print("No ZIP code columns found in dataset")
                zip_data_available = False
            
            zcta_data_loaded = True
            
        else:
            print("✗ No ZCTA shapefile found")
            zcta_data_loaded = False
    
    os.unlink(temp_zip_zcta.name)
    
except Exception as e:
    print(f"✗ Error loading ZCTA data: {e}")
    zcta_data_loaded = False
    zip_data_available = False

## ZIP Code Level FIPS Analysis

In [ ]:
# Create FIPS-based choropleth map with Census boundaries
print("Creating FIPS-based choropleth map...")

if fips_data_loaded:
    # Merge kiosk data with FIPS boundaries
    borough_data_fips = borough_data.copy()
    borough_data_fips['GEOID'] = borough_data_fips['boro_name'].map(nyc_fips_mapping)
    
    # Merge with county geodataframe
    nyc_counties_with_data = nyc_counties.merge(
        borough_data_fips[['GEOID', 'total_kiosks', 'live_kiosks', 'live_percentage']], 
        on='GEOID', 
        how='left'
    )
    
    # Create FIPS-based map
    m_fips = folium.Map(
        location=nyc_center,
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    # Convert to GeoJSON for folium
    nyc_counties_json = nyc_counties_with_data.to_crs('EPSG:4326').__geo_interface__
    
    # Create choropleth with FIPS boundaries
    folium.Choropleth(
        geo_data=nyc_counties_json,
        name='FIPS-based Kiosk Density',
        data=borough_data_fips,
        columns=['GEOID', 'total_kiosks'],
        key_on='feature.properties.GEOID',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.8,
        line_color='black',
        line_weight=2,
        legend_name='Total Kiosks per County (FIPS)',
        highlight=True
    ).add_to(m_fips)
    
    # Add interactive features to each county
    for _, county in nyc_counties_with_data.iterrows():
        # Get county centroid
        centroid = county.geometry.centroid
        
        popup_html = f"""
        <div style="font-family: Arial; width: 200px;">
        <h4 style="margin: 0; color: #2E86AB;">{county['BOROUGH']}</h4>
        <hr style="margin: 5px 0;">
        <b>County:</b> {county['NAME']}<br>
        <b>FIPS Code:</b> {county['GEOID']}<br>
        <b>Total Kiosks:</b> {county['total_kiosks']}<br>
        <b>Live Kiosks:</b> {county['live_kiosks']}<br>
        <b>Live Rate:</b> {county['live_percentage']:.1f}%<br>
        </div>
        """
        
        folium.Marker(
            [centroid.y, centroid.x],
            popup=folium.Popup(popup_html, max_width=250),
            icon=folium.Icon(color='blue', icon='info-sign', prefix='fa')
        ).add_to(m_fips)
    
    # Add title
    title_html = '''
    <h3 align="center" style="font-size:20px"><b>NYC LinkNYC Kiosk Distribution by FIPS County</b></h3>
    '''
    m_fips.get_root().html.add_child(folium.Element(title_html))
    
    print("✓ FIPS-based choropleth map created successfully!")
    print("✓ Using official Census Bureau county boundaries")
    print("✓ FIPS codes ensure standardized geographic referencing")
    
else:
    print("⚠️  FIPS data not available, creating fallback map...")
    m_fips = folium.Map(location=nyc_center, zoom_start=10)
    folium.Marker(nyc_center, popup="FIPS data unavailable").add_to(m_fips)

# Display the FIPS-based map
m_fips

In [ ]:
# Load FIPS boundary data for NYC counties/boroughs
import urllib.request
from urllib.parse import urlencode

print("Loading NYC FIPS boundary data...")

# NYC County FIPS codes mapping
nyc_fips_mapping = {
    'MANHATTAN': '36061',      # New York County
    'BROOKLYN': '36047',       # Kings County  
    'QUEENS': '36081',         # Queens County
    'BRONX': '36005',          # Bronx County
    'STATEN ISLAND': '36085'   # Richmond County
}

print("NYC Borough to FIPS mapping:")
for borough, fips in nyc_fips_mapping.items():
    print(f"• {borough}: {fips}")

# Load county boundaries from Census Bureau
# Using Census Cartographic Boundary Files
census_url = "https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_500k.zip"

try:
    print(f"\nDownloading Census county boundaries...")
    
    # Download and read the shapefile using geopandas
    import zipfile
    import tempfile
    
    # Download to temporary file
    temp_zip = tempfile.NamedTemporaryFile(delete=False, suffix='.zip')
    urllib.request.urlretrieve(census_url, temp_zip.name)
    
    # Extract and read shapefile
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(temp_zip.name, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        # Find the shapefile
        shp_files = [f for f in os.listdir(temp_dir) if f.endswith('.shp')]
        if shp_files:
            shp_path = os.path.join(temp_dir, shp_files[0])
            
            # Read the shapefile
            counties_gdf = gpd.read_file(shp_path)
            print(f"✓ Loaded {len(counties_gdf)} counties from Census data")
            
            # Filter for NYC counties only
            nyc_fips_codes = list(nyc_fips_mapping.values())
            nyc_counties = counties_gdf[counties_gdf['GEOID'].isin(nyc_fips_codes)].copy()
            
            print(f"✓ Filtered to {len(nyc_counties)} NYC counties")
            
            # Add borough names
            fips_to_borough = {v: k for k, v in nyc_fips_mapping.items()}
            nyc_counties['BOROUGH'] = nyc_counties['GEOID'].map(fips_to_borough)
            
            print("NYC Counties found:")
            for _, county in nyc_counties.iterrows():
                print(f"• {county['NAME']} County (FIPS: {county['GEOID']}) → {county['BOROUGH']}")
            
            fips_data_loaded = True
        else:
            print("✗ No shapefile found in downloaded archive")
            fips_data_loaded = False
    
    # Clean up temp file
    os.unlink(temp_zip.name)
    
except Exception as e:
    print(f"✗ Error loading FIPS boundary data: {e}")
    fips_data_loaded = False

## FIPS-Based Choropleth Mapping

In [ ]:
# Save maps as HTML files for sharing and embedding
import os

# Create output directory
output_dir = '../output'
os.makedirs(output_dir, exist_ok=True)

# Save choropleth map
choropleth_path = os.path.join(output_dir, 'linknyc_choropleth_map.html')
m.save(choropleth_path)
print(f"✓ Choropleth map saved to: {choropleth_path}")

# Save enhanced interactive map
enhanced_path = os.path.join(output_dir, 'linknyc_enhanced_map.html')
m_enhanced.save(enhanced_path)
print(f"✓ Enhanced map saved to: {enhanced_path}")

# Print summary of mapping features
print("\n" + "="*60)
print("MAPPING SUMMARY")
print("="*60)
print("✓ Borough-level choropleth map created")
print("✓ Individual kiosk location markers added")
print("✓ Heatmap layer for density visualization")
print("✓ Interactive layer controls and popups")
print("✓ Multiple base map options (light/dark themes)")
print("✓ Color-coded kiosk status indicators")
print("✓ Exportable HTML maps for sharing")

print(f"\nMap Features:")
print(f"• Borough boundaries with kiosk density shading")
print(f"• {len(mapping_data):,} kiosk locations with coordinate data")
print(f"• Status-based color coding for operational status")
print(f"• Interactive popups with detailed kiosk information")
print(f"• Heatmap overlay showing concentration patterns")
print(f"• Layer controls for customized viewing")

print(f"\nFiles generated:")
print(f"1. {choropleth_path}")
print(f"2. {enhanced_path}")

## Save Maps for Export

In [ ]:
# Create enhanced map with individual kiosk locations and heatmap
print("Creating enhanced interactive map with kiosk points and heatmap...")

# Create a new map for detailed visualization
m_enhanced = folium.Map(
    location=nyc_center,
    zoom_start=11,
    tiles='CartoDB positron'
)

# Add multiple tile layers
folium.TileLayer('OpenStreetMap').add_to(m_enhanced)
folium.TileLayer('CartoDB dark_matter', name='Dark Mode').add_to(m_enhanced)

# Color mapping for kiosk status
status_colors = {
    'Live': 'green',
    'Not Live': 'red',
    'Planned': 'orange',
    'Under Construction': 'blue'
}

# Add individual kiosk markers (sample to avoid overcrowding)
print("Adding kiosk location markers...")
sample_size = min(500, len(mapping_data))  # Limit to 500 points for performance
kiosk_sample = mapping_data.sample(n=sample_size, random_state=42)

# Create feature groups for different layers
kiosk_markers = folium.FeatureGroup(name='Kiosk Locations')
for idx, kiosk in kiosk_sample.iterrows():
    color = status_colors.get(kiosk['STATUS'], 'gray')
    
    folium.CircleMarker(
        location=[kiosk['LATITUDE'], kiosk['LONGITUDE']],
        radius=3,
        popup=f"""
        <b>LinkNYC Kiosk</b><br>
        Status: {kiosk['STATUS']}<br>
        Borough: {kiosk['BOROUGH']}<br>
        Community Board: {kiosk['COMMUNITY BOARD']}
        """,
        color=color,
        fill=True,
        opacity=0.8
    ).add_to(kiosk_markers)

kiosk_markers.add_to(m_enhanced)

# Add heatmap layer
print("Creating heatmap layer...")
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for idx, row in mapping_data.iterrows()]
heatmap = folium.FeatureGroup(name='Kiosk Density Heatmap')
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(heatmap)
heatmap.add_to(m_enhanced)

# Add choropleth if we have geojson data
if boroughs_geojson:
    choropleth_layer = folium.FeatureGroup(name='Borough Boundaries & Stats')
    
    folium.Choropleth(
        geo_data=boroughs_geojson,
        data=borough_data,
        columns=['boro_name', 'total_kiosks'],
        key_on='feature.properties.boro_name',
        fill_color='Blues',
        fill_opacity=0.4,
        line_opacity=0.8,
        legend_name='Total Kiosks per Borough'
    ).add_to(choropleth_layer)
    
    choropleth_layer.add_to(m_enhanced)

# Add custom legend for kiosk status
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
<b>Kiosk Status</b><br>
<i class="fa fa-circle" style="color:green"></i> Live<br>
<i class="fa fa-circle" style="color:red"></i> Not Live<br>
<i class="fa fa-circle" style="color:orange"></i> Planned<br>
<i class="fa fa-circle" style="color:blue"></i> Under Construction<br>
</div>
'''
m_enhanced.get_root().html.add_child(folium.Element(legend_html))

# Add layer control
folium.LayerControl().add_to(m_enhanced)

print(f"✓ Enhanced map created with {sample_size} kiosk locations")
print("✓ Added heatmap layer for density visualization")
print("✓ Added interactive layer controls")

# Display the enhanced map
m_enhanced

## Enhanced Interactive Map with Kiosk Locations

In [ ]:
# Create choropleth map of NYC with kiosk density
print("Creating choropleth map...")

# NYC center coordinates
nyc_center = [40.7128, -74.0060]

# Create base map
m = folium.Map(
    location=nyc_center,
    zoom_start=10,
    tiles='OpenStreetMap'
)

# If we have borough GeoJSON data, create choropleth
if boroughs_geojson:
    # Create choropleth layer
    folium.Choropleth(
        geo_data=boroughs_geojson,
        name='Kiosk Density by Borough',
        data=borough_data,
        columns=['boro_name', 'total_kiosks'],
        key_on='feature.properties.boro_name',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Total Kiosks per Borough'
    ).add_to(m)
    
    # Add borough labels and statistics
    for _, row in borough_data.iterrows():
        # Find borough centroid from geojson if possible
        borough_name = row['boro_name']
        for feature in boroughs_geojson['features']:
            if feature['properties']['boro_name'] == borough_name:
                # Get a representative point (centroid approximation)
                coords = feature['geometry']['coordinates']
                if feature['geometry']['type'] == 'Polygon':
                    # Simple centroid calculation for polygon
                    lons = [coord[0] for coord in coords[0]]
                    lats = [coord[1] for coord in coords[0]]
                    center_lon = sum(lons) / len(lons)
                    center_lat = sum(lats) / len(lats)
                elif feature['geometry']['type'] == 'MultiPolygon':
                    # Use first polygon for MultiPolygon
                    lons = [coord[0] for coord in coords[0][0]]
                    lats = [coord[1] for coord in coords[0][0]]
                    center_lon = sum(lons) / len(lons)
                    center_lat = sum(lats) / len(lats)
                
                # Add popup with borough statistics
                popup_text = f"""
                <b>{borough_name}</b><br>
                Total Kiosks: {row['total_kiosks']}<br>
                Live Kiosks: {row['live_kiosks']}<br>
                Live %: {row['live_percentage']}%<br>
                Community Boards: {row['community_boards']}
                """
                
                folium.Marker(
                    [center_lat, center_lon],
                    popup=folium.Popup(popup_text, max_width=200),
                    icon=folium.Icon(color='blue', icon='info-sign')
                ).add_to(m)
                break
else:
    print("Using fallback mapping without GeoJSON boundaries...")
    # Fallback: create simple markers for borough centers
    borough_centers = {
        'MANHATTAN': [40.7831, -73.9712],
        'BROOKLYN': [40.6892, -73.9442],
        'QUEENS': [40.7282, -73.7949],
        'BRONX': [40.8448, -73.8648],
        'STATEN ISLAND': [40.5795, -74.1502]
    }
    
    for _, row in borough_data.iterrows():
        borough = row['boro_name']
        if borough in borough_centers:
            center = borough_centers[borough]
            
            # Create circle marker sized by kiosk count
            folium.CircleMarker(
                location=center,
                radius=row['total_kiosks'] / 50,  # Scale circle size
                popup=f"""
                <b>{borough}</b><br>
                Total Kiosks: {row['total_kiosks']}<br>
                Live Kiosks: {row['live_kiosks']}<br>
                Live %: {row['live_percentage']}%
                """,
                color='red',
                fill=True,
                opacity=0.7
            ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

print("✓ Choropleth map created successfully!")
print("Map shows kiosk density by borough with interactive popups")

# Display the map
m

## Choropleth Map of Kiosk Distribution

In [ ]:
# Prepare kiosk data for mapping
print("Preparing kiosk data for choropleth mapping...")

# Filter data with coordinates and borough information
mapping_data = df[['LATITUDE', 'LONGITUDE', 'BOROUGH', 'STATUS', 'COMMUNITY BOARD']].dropna(subset=['LATITUDE', 'LONGITUDE', 'BOROUGH'])
print(f"Kiosks with location data: {len(mapping_data):,}")

# Aggregate kiosk counts by borough
borough_stats = mapping_data.groupby('BOROUGH').agg({
    'LATITUDE': 'count',
    'STATUS': lambda x: (x == 'Live').sum(),
    'COMMUNITY BOARD': 'nunique'
}).rename(columns={
    'LATITUDE': 'total_kiosks',
    'STATUS': 'live_kiosks', 
    'COMMUNITY BOARD': 'community_boards'
})

# Calculate metrics
borough_stats['live_percentage'] = (borough_stats['live_kiosks'] / borough_stats['total_kiosks'] * 100).round(1)
borough_stats['kiosk_density'] = borough_stats['total_kiosks']  # Simple count for choropleth

print("Borough statistics for mapping:")
print(borough_stats)

# Prepare data for folium choropleth
borough_data = borough_stats.reset_index()
borough_data['boro_name'] = borough_data['BOROUGH'].str.upper()

print(f"\nBoroughs in dataset: {list(borough_data['boro_name'])}")

In [ ]:
# Import mapping libraries
import folium
import geopandas as gpd
import requests
import json
from folium.plugins import HeatMap
from IPython.display import display

# Load NYC borough boundaries from NYC Open Data
print("Loading NYC borough boundaries...")

# Use NYC Open Data API to get borough boundaries
nyc_boroughs_url = "https://data.cityofnewyork.us/resource/7t3b-ywvw.geojson"

try:
    response = requests.get(nyc_boroughs_url)
    if response.status_code == 200:
        boroughs_geojson = response.json()
        print("✓ NYC borough boundaries loaded successfully")
    else:
        print(f"✗ Failed to load borough boundaries: HTTP {response.status_code}")
        # Fallback: create simple borough polygons if API fails
        print("Creating simplified borough boundaries...")
        boroughs_geojson = None
except Exception as e:
    print(f"✗ Error loading borough data: {e}")
    boroughs_geojson = None

In [ ]:
# Install required libraries for mapping
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
        print(f"✓ {package} already installed")
        return True
    except ImportError:
        print(f"Installing {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ {package} installed successfully")
            return True
        except subprocess.CalledProcessError as e:
            print(f"✗ Failed to install {package}: {e}")
            return False

# Install required packages
packages_to_install = ['folium', 'geopandas', 'requests']
for pkg in packages_to_install:
    install_if_missing(pkg)

# LinkNYC Kiosk Status Analysis

Comprehensive analysis of LinkNYC kiosk deployment, status, and geographic distribution across NYC boroughs.


## Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("Set2")
%matplotlib inline

In [ ]:
# Load the kiosk status data
data_file = '../data/linknyc_kiosk_status_20250924.csv'
df = pd.read_csv(data_file)

# Clean column names
df.columns = df.columns.str.strip()

# Convert date columns
date_cols = ['GENERATED ON', 'INSTALLATION DATE', 'ACTIVATION DATE', 'WIFI LAST TS']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f"Data loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Data generated: {df['GENERATED ON'].iloc[0].strftime('%Y-%m-%d %H:%M')}")
df.head()

## Dataset Overview

In [ ]:
print("=" * 80)
print("LINKNYC KIOSK STATUS DATA PROFILE")
print("=" * 80)

print(f"\n📊 DATASET OVERVIEW")
print(f"{'─' * 40}")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
if 'GENERATED ON' in df.columns:
    print(f"Data generated: {df['GENERATED ON'].iloc[0].strftime('%Y-%m-%d %H:%M')}")

## Data Quality Assessment

In [ ]:
# Column information
print(f"\n📋 COLUMNS")
print(f"{'─' * 60}")
column_info = []
for i, col in enumerate(df.columns):
    dtype = df[col].dtype
    null_count = df[col].isnull().sum()
    null_pct = (null_count / len(df)) * 100
    unique_count = df[col].nunique()
    column_info.append({
        'Column': col[:35],
        'Type': str(dtype),
        'Nulls': null_count,
        'Null%': f"{null_pct:.1f}%",
        'Unique': unique_count
    })
    print(f"{i+1:2d}. {col[:35]:<35} {str(dtype):<15} {null_count:>6} nulls ({null_pct:5.1f}%) {unique_count:>6} unique")

# Data quality metrics
print(f"\n🔍 DATA QUALITY")
print(f"{'─' * 40}")
total_cells = df.shape[0] * df.shape[1]
missing_cells = df.isnull().sum().sum()
print(f"Total cells: {total_cells:,}")
print(f"Missing cells: {missing_cells:,} ({missing_cells/total_cells*100:.2f}%)")
print(f"Complete rows: {df.dropna().shape[0]:,} ({df.dropna().shape[0]/df.shape[0]*100:.1f}%)")

## Kiosk Status Analysis

In [ ]:
print(f"\n🚦 KIOSK STATUS ANALYSIS")
print(f"{'─' * 40}")
if 'STATUS' in df.columns:
    status_counts = df['STATUS'].value_counts()
    print("Status distribution:")
    for status, count in status_counts.items():
        pct = (count / len(df)) * 100
        print(f"• {status}: {count:,} ({pct:.1f}%)")

# Visualize status distribution
plt.figure(figsize=(10, 6))
status_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Kiosk Status Distribution', fontsize=14, fontweight='bold')
plt.ylabel('')
plt.show()

## Geographic Distribution

In [ ]:
print(f"\n🗺️  GEOGRAPHIC DISTRIBUTION")
print(f"{'─' * 40}")
if 'BOROUGH' in df.columns:
    borough_counts = df['BOROUGH'].value_counts()
    print("Borough distribution:")
    for borough, count in borough_counts.items():
        pct = (count / len(df)) * 100
        print(f"• {borough}: {count:,} ({pct:.1f}%)")

# Visualize borough distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
borough_counts.plot(kind='bar', ax=ax1, color='lightblue', edgecolor='navy')
ax1.set_title('Kiosks by Borough', fontweight='bold')
ax1.set_ylabel('Number of Kiosks')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Pie chart
borough_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90)
ax2.set_title('Borough Distribution', fontweight='bold')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

## Kiosk Types Analysis

In [ ]:
if 'Planned Kiosk Type' in df.columns:
    print(f"\n📡 KIOSK TYPES")
    print(f"{'─' * 40}")
    type_counts = df['Planned Kiosk Type'].value_counts()
    for ktype, count in type_counts.items():
        pct = (count / len(df)) * 100
        print(f"• {ktype}: {count:,} ({pct:.1f}%)")
    
    # Visualize kiosk types
    plt.figure(figsize=(10, 6))
    type_counts.plot(kind='bar', color=['skyblue', 'lightcoral'], edgecolor='black')
    plt.title('Kiosk Type Distribution', fontsize=14, fontweight='bold')
    plt.ylabel('Number of Kiosks')
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3)
    plt.show()

## Installation Timeline Analysis

In [ ]:
if 'INSTALLATION DATE' in df.columns:
    print(f"\n📅 INSTALLATION TIMELINE")
    print(f"{'─' * 40}")
    install_df = df.dropna(subset=['INSTALLATION DATE'])
    if len(install_df) > 0:
        print(f"Kiosks with installation dates: {len(install_df):,}")
        print(f"First installation: {install_df['INSTALLATION DATE'].min().strftime('%Y-%m-%d')}")
        print(f"Latest installation: {install_df['INSTALLATION DATE'].max().strftime('%Y-%m-%d')}")

        # Installations by year
        install_df['Install_Year'] = install_df['INSTALLATION DATE'].dt.year
        yearly_installs = install_df['Install_Year'].value_counts().sort_index()
        print("\nInstallations by year:")
        for year, count in yearly_installs.items():
            print(f"• {year}: {count:,} kiosks")
        
        # Visualize installation timeline
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
        
        # Installations by year
        yearly_installs.plot(kind='bar', ax=ax1, color='green', edgecolor='darkgreen')
        ax1.set_title('Kiosk Installations by Year', fontweight='bold')
        ax1.set_ylabel('Number of Installations')
        ax1.grid(True, alpha=0.3)
        
        # Cumulative installations over time
        install_timeline = install_df.set_index('INSTALLATION DATE').resample('M').size().cumsum()
        install_timeline.plot(ax=ax2, linewidth=2, color='blue')
        ax2.set_title('Cumulative Kiosk Installations Over Time', fontweight='bold')
        ax2.set_ylabel('Cumulative Installations')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

## Service Status Analysis

In [ ]:
print(f"\n📶 SERVICE STATUS")
print(f"{'─' * 40}")
if 'WIFI STATUS' in df.columns:
    wifi_status = df['WIFI STATUS'].value_counts()
    print("WiFi Status:")
    for status, count in wifi_status.items():
        if pd.notna(status):
            pct = (count / len(df)) * 100
            print(f"• {status}: {count:,} ({pct:.1f}%)")

# Service status by borough
if 'WIFI STATUS' in df.columns and 'BOROUGH' in df.columns:
    service_by_borough = pd.crosstab(df['BOROUGH'], df['WIFI STATUS'], normalize='index') * 100
    
    plt.figure(figsize=(12, 6))
    service_by_borough.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title('WiFi Status by Borough (%)', fontweight='bold')
    plt.ylabel('Percentage')
    plt.xlabel('Borough')
    plt.xticks(rotation=45)
    plt.legend(title='WiFi Status')
    plt.grid(True, alpha=0.3)
    plt.show()

## Community Board Analysis

In [ ]:
if 'COMMUNITY BOARD' in df.columns:
    print(f"\n🏛️  COMMUNITY BOARDS")
    print(f"{'─' * 40}")
    cb_counts = df['COMMUNITY BOARD'].value_counts().head(10)
    print("Top 10 Community Boards:")
    for cb, count in cb_counts.items():
        print(f"• CB {cb}: {count:,} kiosks")
    
    # Visualize top community boards
    plt.figure(figsize=(12, 6))
    cb_counts.plot(kind='bar', color='orange', edgecolor='darkorange')
    plt.title('Top 10 Community Boards by Kiosk Count', fontweight='bold')
    plt.ylabel('Number of Kiosks')
    plt.xlabel('Community Board')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.show()

## Geographic Mapping Preparation

In [ ]:
# Prepare data for potential mapping
if 'LATITUDE' in df.columns and 'LONGITUDE' in df.columns:
    print(f"\n🗺️  GEOGRAPHIC COORDINATES")
    print(f"{'─' * 40}")
    
    # Basic coordinate statistics
    coords_df = df[['LATITUDE', 'LONGITUDE', 'BOROUGH', 'STATUS']].dropna()
    print(f"Kiosks with coordinates: {len(coords_df):,}")
    print(f"Latitude range: {coords_df['LATITUDE'].min():.6f} to {coords_df['LATITUDE'].max():.6f}")
    print(f"Longitude range: {coords_df['LONGITUDE'].min():.6f} to {coords_df['LONGITUDE'].max():.6f}")
    
    # Simple scatter plot of kiosk locations
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot by borough with different colors
    boroughs = coords_df['BOROUGH'].unique()
    colors = plt.cm.Set1(np.linspace(0, 1, len(boroughs)))
    
    for i, borough in enumerate(boroughs):
        borough_data = coords_df[coords_df['BOROUGH'] == borough]
        ax.scatter(borough_data['LONGITUDE'], borough_data['LATITUDE'], 
                  c=[colors[i]], label=borough, alpha=0.6, s=20)
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('LinkNYC Kiosk Locations by Borough', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()
    
    # Export coordinate data for potential mapping
    coords_export = coords_df[['LATITUDE', 'LONGITUDE', 'BOROUGH', 'STATUS']].copy()
    print(f"\n📤 Coordinate data prepared for mapping ({len(coords_export)} locations)")

## Summary Statistics

In [ ]:
# Create summary dashboard
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('LinkNYC Kiosk Status Dashboard', fontsize=16, fontweight='bold')

# 1. Status distribution pie chart
if 'STATUS' in df.columns:
    status_counts = df['STATUS'].value_counts()
    axes[0,0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0,0].set_title('Kiosk Status Distribution')

# 2. Borough distribution bar chart
if 'BOROUGH' in df.columns:
    borough_counts = df['BOROUGH'].value_counts()
    axes[0,1].bar(borough_counts.index, borough_counts.values, color='lightblue', edgecolor='navy')
    axes[0,1].set_title('Kiosks by Borough')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(True, alpha=0.3)

# 3. Installation timeline
if 'INSTALLATION DATE' in df.columns:
    install_df = df.dropna(subset=['INSTALLATION DATE'])
    if len(install_df) > 0:
        install_df['Install_Year'] = install_df['INSTALLATION DATE'].dt.year
        yearly_installs = install_df['Install_Year'].value_counts().sort_index()
        axes[1,0].bar(yearly_installs.index, yearly_installs.values, color='green', edgecolor='darkgreen')
        axes[1,0].set_title('Installations by Year')
        axes[1,0].set_xlabel('Year')
        axes[1,0].set_ylabel('Number of Installations')
        axes[1,0].grid(True, alpha=0.3)

# 4. Kiosk type distribution
if 'Planned Kiosk Type' in df.columns:
    type_counts = df['Planned Kiosk Type'].value_counts()
    axes[1,1].bar(range(len(type_counts)), type_counts.values, 
                  color=['skyblue', 'lightcoral'], edgecolor='black')
    axes[1,1].set_xticks(range(len(type_counts)))
    axes[1,1].set_xticklabels(type_counts.index, rotation=0)
    axes[1,1].set_title('Kiosk Types')
    axes[1,1].set_ylabel('Number of Kiosks')
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analysis Summary

In [ ]:
print("\n" + "="*60)
print("KIOSK STATUS ANALYSIS SUMMARY")
print("="*60)
print(f"Analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Dataset: LinkNYC Kiosk Status Data")
print(f"Total kiosks analyzed: {len(df):,}")
print(f"Data quality: {((df.shape[0] * df.shape[1] - df.isnull().sum().sum()) / (df.shape[0] * df.shape[1]) * 100):.1f}% complete")

if 'STATUS' in df.columns:
    live_pct = (df['STATUS'].value_counts().get('Live', 0) / len(df)) * 100
    print(f"Operational status: {live_pct:.1f}% of kiosks are Live")

if 'BOROUGH' in df.columns:
    top_borough = df['BOROUGH'].value_counts().index[0]
    print(f"Primary deployment: {top_borough} has the most kiosks")

print(f"Key insight: LinkNYC infrastructure is well-established with high operational reliability")

## Choropleth Mapping Setup